In [2]:
import torch as th
import numpy as np

In [5]:
import pickle

# 定义新文件路径
goodreads_children_path = 'Dataset/goodreads_children/processed/children.pkl'

# 加载 goodreads_children.pkl 文件
with open(goodreads_children_path, 'rb') as f:
    goodreads_children_data = pickle.load(f)

# 打印加载的数据
print(goodreads_children_data)

Data(edge_index=[2, 734640], text_nodes=[216613], text_edges=[734640], text_node_labels=[216613], edge_score=[734640])


In [7]:
product_labels = goodreads_children_data.text_node_labels


In [9]:
from sklearn.preprocessing import MultiLabelBinarizer

In [10]:
mlb = MultiLabelBinarizer()
binary_labels = mlb.fit_transform(product_labels)

TypeError: 'int' object is not iterable

In [64]:
dgl_graph = goodreads_children_data
test_ratio=0.1
val_ratio=0.2
random_seed=42
neg_len="1000"
path=None
way="random"

In [65]:
graph = dgl_graph

eids = np.arange(graph.num_edges)
eids = np.random.permutation(eids)

u, v = graph.edge_index

test_size = int(len(eids) * test_ratio)
val_size = int(len(eids) * val_ratio)

test_pos_u, test_pos_v = u[eids[:test_size]], v[eids[:test_size]]
val_pos_u, val_pos_v = (
	u[eids[test_size : test_size + val_size]],
	v[eids[test_size : test_size + val_size]],
)
train_pos_u, train_pos_v = (
	u[eids[test_size + val_size :]],
	v[eids[test_size + val_size :]],
)

valid_target_neg = th.randint(
	0, graph.num_nodes, [len(val_pos_u), int(neg_len)], dtype=th.long
)  # TODO: filter some edges in graph
test_target_neg = th.randint(
	0, graph.num_nodes, [len(test_pos_u), int(neg_len)], dtype=th.long
)  # TODO: filter some edges in graph



/home/lizhuofeng/.local/lib/python3.10/site-packages/torch_geometric/data/storage.py:452: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'edge_index', 'text_edges', 'text_nodes'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(


In [76]:
from torch_sparse import SparseTensor


adj_t = SparseTensor.from_edge_index(goodreads_children_data.edge_index, sparse_sizes=(goodreads_children_data.num_nodes, goodreads_children_data.num_nodes)).t()


In [73]:
goodreads_children_data.num_nodes

/home/lizhuofeng/.local/lib/python3.10/site-packages/torch_geometric/data/storage.py:452: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'edge_index', 'text_edges', 'text_nodes'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(


60785

In [77]:
adj_t.size(0), adj_t.size(1)

(60785, 60785)

In [2]:
from torch.utils.data import DataLoader

In [3]:
for perm in DataLoader(range(10), 2, shuffle=True):
    print(perm)

tensor([4, 9])
tensor([6, 8])
tensor([0, 5])
tensor([7, 3])
tensor([1, 2])


In [2]:
import copy
import os.path as osp
import time

import torch
import torch.nn.functional as F
from tqdm import tqdm

from torch_geometric.datasets import Reddit
from torch_geometric.loader import NeighborLoader
from torch_geometric.nn import SAGEConv

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

dataset = Reddit("Reddit")

# Already send node features/labels to GPU for faster access during sampling:
data = dataset[0].to(device, 'x', 'y')

kwargs = {'batch_size': 1024, 'num_workers': 6, 'persistent_workers': True}
train_loader = NeighborLoader(data, input_nodes=data.train_mask,
                              num_neighbors=[25, 10], shuffle=True, **kwargs)

subgraph_loader = NeighborLoader(copy.copy(data), input_nodes=None,
                                 num_neighbors=[-1], shuffle=False, **kwargs)

Extracting ./raw/reddit.zip
Processing...
Done!


In [17]:
import copy
import os.path as osp
import time

import torch
import torch.nn.functional as F
from tqdm import tqdm

from torch_geometric.datasets import Reddit
from torch_geometric.loader import NeighborLoader
from torch_geometric.nn import SAGEConv

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# path = osp.join(osp.dirname(osp.realpath(__file__)), '..', 'data', 'Reddit')
dataset = Reddit("Reddit")

# Already send node features/labels to GPU for faster access during sampling:
data = dataset[0].to(device, 'x', 'y')

kwargs = {'batch_size': 1024, 'num_workers': 6, 'persistent_workers': True}
train_loader = NeighborLoader(data, input_nodes=data.train_mask,
                              num_neighbors=[25, 10], shuffle=True, **kwargs)

subgraph_loader = NeighborLoader(copy.copy(data), input_nodes=None,
                                 num_neighbors=[-1], shuffle=False, **kwargs)

# No need to maintain these features during evaluation:
del subgraph_loader.data.x, subgraph_loader.data.y
# Add global node index information.
subgraph_loader.data.num_nodes = data.num_nodes
subgraph_loader.data.n_id = torch.arange(data.num_nodes)


class SAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.convs = torch.nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        self.convs.append(SAGEConv(hidden_channels, out_channels))

    def forward(self, x, edge_index):
        for i, conv in enumerate(self.convs):
            x = conv(x, edge_index)
            if i < len(self.convs) - 1:
                x = x.relu_()
                x = F.dropout(x, p=0.5, training=self.training)
        return x

    @torch.no_grad()
    def inference(self, x_all, subgraph_loader):
        pbar = tqdm(total=len(subgraph_loader.dataset) * len(self.convs))
        pbar.set_description('Evaluating')

        # Compute representations of nodes layer by layer, using *all*
        # available edges. This leads to faster computation in contrast to
        # immediately computing the final representations of each batch:
        for i, conv in enumerate(self.convs):
            xs = []
            for batch in subgraph_loader:
                x = x_all[batch.n_id.to(x_all.device)].to(device)
                x = conv(x, batch.edge_index.to(device))
                if i < len(self.convs) - 1:
                    x = x.relu_()
                xs.append(x[:batch.batch_size].cpu())
                pbar.update(batch.batch_size)
            x_all = torch.cat(xs, dim=0)
        pbar.close()
        return x_all


model = SAGE(dataset.num_features, 256, dataset.num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)


def train(epoch):
    model.train()

    pbar = tqdm(total=int(len(train_loader.dataset)))
    pbar.set_description(f'Epoch {epoch:02d}')

    total_loss = total_correct = total_examples = 0
    for batch in train_loader:
        optimizer.zero_grad()
        print(batch.y.shape, batch.x.shape, batch.batch_size)
        y = batch.y[:batch.batch_size]
        y_hat = model(batch.x, batch.edge_index.to(device))[:batch.batch_size]
        loss = F.cross_entropy(y_hat, y)
        loss.backward()
        optimizer.step()

        total_loss += float(loss) * batch.batch_size
        total_correct += int((y_hat.argmax(dim=-1) == y).sum())
        total_examples += batch.batch_size
        pbar.update(batch.batch_size)
    pbar.close()

    return total_loss / total_examples, total_correct / total_examples


@torch.no_grad()
def test():
    model.eval()
    y_hat = model.inference(data.x, subgraph_loader).argmax(dim=-1)
    y = data.y.to(y_hat.device)

    accs = []
    for mask in [data.train_mask, data.val_mask, data.test_mask]:
        accs.append(int((y_hat[mask] == y[mask]).sum()) / int(mask.sum()))
    return accs


times = []
for epoch in range(1, 11):
    start = time.time()
    loss, acc = train(epoch)
    print(f'Epoch {epoch:02d}, Loss: {loss:.4f}, Approx. Train: {acc:.4f}')
    train_acc, val_acc, test_acc = test()
    print(f'Epoch: {epoch:02d}, Train: {train_acc:.4f}, Val: {val_acc:.4f}, '
          f'Test: {test_acc:.4f}')
    times.append(time.time() - start)
print(f"Median time per epoch: {torch.tensor(times).median():.4f}s")

torch.Size([106735]) torch.Size([106735, 602]) 1024
torch.Size([105481]) torch.Size([105481, 602]) 1024
torch.Size([105129]) torch.Size([105129, 602]) 1024
torch.Size([106220]) torch.Size([106220, 602]) 1024
torch.Size([105261]) torch.Size([105261, 602]) 1024
torch.Size([106320]) torch.Size([106320, 602]) 1024


torch.Size([105405]) torch.Size([105405, 602]) 1024
torch.Size([106596]) torch.Size([106596, 602]) 1024
torch.Size([105676]) torch.Size([105676, 602]) 1024
torch.Size([104610]) torch.Size([104610, 602]) 1024
torch.Size([106998]) torch.Size([106998, 602]) 1024
torch.Size([106631]) torch.Size([106631, 602]) 1024
torch.Size([105737]) torch.Size([105737, 602]) 1024


torch.Size([105855]) torch.Size([105855, 602]) 1024
torch.Size([105222]) torch.Size([105222, 602]) 1024
torch.Size([105959]) torch.Size([105959, 602]) 1024
torch.Size([105581]) torch.Size([105581, 602]) 1024
torch.Size([106453]) torch.Size([106453, 602]) 1024
torch.Size([105209]) torch.Size([105209, 602]) 1024
torch.Size([105987]) torch.Size([105987, 602]) 1024


torch.Size([105805]) torch.Size([105805, 602]) 1024
torch.Size([105909]) torch.Size([105909, 602]) 1024
torch.Size([105238]) torch.Size([105238, 602]) 1024
torch.Size([105930]) torch.Size([105930, 602]) 1024
torch.Size([105840]) torch.Size([105840, 602]) 1024
torch.Size([105641]) torch.Size([105641, 602]) 1024
torch.Size([105828]) torch.Size([105828, 602]) 1024
torch.Size([105311]) torch.Size([105311, 602]) 1024


torch.Size([105526]) torch.Size([105526, 602]) 1024
torch.Size([105432]) torch.Size([105432, 602]) 1024
torch.Size([105646]) torch.Size([105646, 602]) 1024
torch.Size([105406]) torch.Size([105406, 602]) 1024
torch.Size([105674]) torch.Size([105674, 602]) 1024
torch.Size([105651]) torch.Size([105651, 602]) 1024
torch.Size([106277]) torch.Size([106277, 602]) 1024
torch.Size([105943]) torch.Size([105943, 602]) 1024


torch.Size([105412]) torch.Size([105412, 602]) 1024
torch.Size([105830]) torch.Size([105830, 602]) 1024
torch.Size([105797]) torch.Size([105797, 602]) 1024
torch.Size([104492]) torch.Size([104492, 602]) 1024
torch.Size([106243]) torch.Size([106243, 602]) 1024
torch.Size([105881]) torch.Size([105881, 602]) 1024
torch.Size([104997]) torch.Size([104997, 602]) 1024
torch.Size([105985]) torch.Size([105985, 602]) 1024


torch.Size([106930]) torch.Size([106930, 602]) 1024
torch.Size([105929]) torch.Size([105929, 602]) 1024


Evaluating:  85%|████████▍ | 395781/465930 [01:48<00:19, 3662.48it/s] 


KeyboardInterrupt: 

In [18]:
import torch
from torch_geometric.data import Data
from torch_geometric.loader import NeighborLoader
from torch_sparse import SparseTensor


x = torch.randn(8, 32)  # Node features of shape [num_nodes, num_features]
y = torch.randint(0, 4, (8, ))  # Node labels of shape [num_nodes]
edge_index = torch.tensor([
    [2, 3, 3, 4, 5, 6, 7],
    [0, 0, 1, 1, 2, 3, 4]],
)
edge_feature = torch.randn(edge_index.shape[1], 16)
adj_t = SparseTensor.from_edge_index(edge_index, edge_feature, sparse_sizes=(data.num_nodes, data.num_nodes)).t()

#   0  1
#  / \/ \
# 2  3  4
# |  |  |
# 5  6  7

data = Data(x=x, y=y, adj_t=adj_t)

loader = NeighborLoader(
    data,
    input_nodes=torch.tensor([0, 1]),
    num_neighbors=[2, 1],
    batch_size=2,
    replace=False,
    shuffle=False,
)

In [ ]:
NeighborSampler

In [19]:
batch = next(iter(loader))

In [20]:
batch

Data(x=[8, 32], y=[8], adj_t=[8, 8, 16, nnz=7], n_id=[8], e_id=[7], num_sampled_nodes=[3], num_sampled_edges=[2], input_id=[2], batch_size=2)

In [21]:
batch.n_id

tensor([0, 1, 2, 3, 4, 5, 6, 7])

In [8]:
batch.y

tensor([1, 3, 0, 2, 0])